In [1]:
vintage_date = '2011-01-01'

In [6]:
import pathlib
import pandas as pd

dir_raw = pathlib.Path('/raw')

In [7]:
dir_raw.exists()

False

In [9]:
pwd

'c:\\Users\\dev\\Documents\\GitHub\\MMB_forecast_application'